In [1]:
def flatten_json(nested_json, exclude=['']):
    """Flatten json object with nested keys into a single level.
        Args:
            nested_json: A nested json object.
            exclude: Keys to exclude from output.
        Returns:
            The flattened json object if successful, None otherwise.
    """
    out = {}

    def flatten(x, name='', exclude=exclude):
        if type(x) is dict:
            for a in x:
                if a not in exclude: flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(nested_json)
    return out

In [88]:
from flatten_json import flatten
import json

with open('Services.json') as f:
    data = json.load(f)
dic_flattened = (flatten(d) for d in data)
df = pd.DataFrame(dic_flattened)
clos = [col for col in list(df.columns) if 'Propertie' not in col]
df = df[clos]#.drop_duplicates().to_csv('cloud_service.csv', index=False)

In [89]:
def rchop(s, suffix):
    if suffix and s.endswith(suffix):
         s = s[:-len(suffix)]
    if suffix and s.endswith(suffix):
        return rchop(s, suffix)
    return s

def concate(df, cloud, type_):
    col_ = [col for col in list(df.columns) if cloud in col and type_  in col]
    return df[col_].fillna('').astype(str).agg('<br/>'.join, axis=1).apply(lambda x: rchop(x, '<br/>') )


clouds = ['aws', 'azure', 'google', 'ibm', 'alibaba', 'oracle']
type_s = ['name', 'ref', 'icon']

for cloud in clouds:
    for  type_ in type_s:
        df[cloud +'_'+ type_] = concate(df, cloud, type_)


In [90]:
df.drop_duplicates().to_csv('cloud_service.csv', index=False)